In [55]:
import utils.common
import evaluation
import importlib
import numpy as np
import tensorflow as tf
import models.MatrixFactorization
import models.BPR
import models.ConstraintAutoRec
import models.NeuralLogicRec
import pandas as pd
from models.NeuralLogicRec import item_cf, user_cf, diversity_constraint, Constraint, And, Or, Implies, Forall, Not, Equiv
importlib.reload(utils.common)
importlib.reload(evaluation)
importlib.reload(models.NeuralLogicRec)

<module 'models.NeuralLogicRec' from '/mnt/e/owncloud/FH/MasterThesis/Code/models/NeuralLogicRec.py'>

In [41]:
ev_test = evaluation.Evaluation(utils.common.ml_small)

In [56]:
constraints = list()
constraints.append(Constraint(weight=0.75, formula=item_cf))
constraints.append(Constraint(weight=0.75, formula=user_cf))
@tf.function
def likes_equiv(model, outputs):
    return Forall(Equiv(outputs['rec'], outputs['likes']))
constraints.append(Constraint(weight=0.75, formula=likes_equiv))
@tf.function
def novelty_constraint(model, outputs):
    return Forall(Implies(outputs['popular'], Not(outputs['rec'])))
constraints.append(Constraint(weight=0.25, formula=novelty_constraint))
constraints.append(Constraint(weight=0.25, formula=diversity_constraint))

In [57]:
nlr = models.NeuralLogicRec.NLR(utils.common.ml_small['user'], utils.common.ml_small['dimensions'], epochs=10, constraints=constraints, mode='ae')
nlr.train(utils.common.load_dataset(utils.common.ml_small), utils.common.ml_small['train']['records'])

/home/markush/anaconda3/envs/tensorflow/lib/python3.7/site-packages/tensorflow_core/python/framework/indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


/home/markush/anaconda3/envs/tensorflow/lib/python3.7/site-packages/tensorflow_core/python/framework/indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Epoch #0 Loss at step 12: 0.4640, time: 73.853
Epoch #1 Loss at step 12: 0.4294, time: 54.535Train accuracy 0.655, Validation accuracy 0.135
Epoch #2 Loss at step 12: 0.3874, time: 54.860Train accuracy 0.671, Validation accuracy 0.229
Epoch #3 Loss at step 12: 0.3940, time: 54.317Train accuracy 0.655, Validation accuracy 0.160
Epoch #4 Loss at step 12: 0.3482, time: 54.629Train accuracy 0.666, Validation accuracy 0.149
Epoch #5 Loss at step 12: 0.3384, time: 54.901Train accuracy 0.657, Validation accuracy 0.214
Epoch #6 Loss at step 12: 0.3077, time: 55.315Train accuracy 0.653, Validation accuracy 0.153
Epoch #7 Loss at step 12: 0.2917, time: 54.365Train accuracy 0.658, Validation accuracy 0.216
Epoch #8 Loss at step 12: 0.2682, time: 55.030Train accuracy 0.650, Validation accuracy 0.186
Epoch #9 Loss at step 12: 0.2727, time: 53.765Train accuracy 0.672, Validation accuracy 0.159


In [53]:
results.append(ev_test.evaluate_single_thread(nlr))